In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [6]:
df = pd.read_csv('swat.csv')
#df.shape

df = df.iloc[:5*len(df.index)//8]
#df = df.iloc[:len(df.index)//2]
#df = df.iloc[:7500]
df.head(5)

,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,1451296800000000000,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,1451296801000000000,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,1451296802000000000,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,1451296803000000000,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,1451296804000000000,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal


In [ ]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """Aggregate values over given time span.
    Args:
        X (ndarray or pandas.DataFrame):
            N-dimensional sequence of values.
        interval (int):
            Integer denoting time span to compute aggregation of.
        time_column (int):
            Column of X that contains time values.
        method (str or list):
            Optional. String describing aggregation method or list of strings describing multiple
            aggregation methods. If not given, `mean` is used.
    Returns:
        ndarray, ndarray:
            * Sequence of aggregated values, one column for each aggregation method.
            * Sequence of index values (first index of each aggregated segment).
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)

    X = X.sort_values(time_column).set_index(time_column)

    if isinstance(method, str):
        method = [method]

    start_ts = X.index.values[0]
    max_ts = X.index.values[-1]

    values = list()
    index = list()
    while start_ts <= max_ts:
        end_ts = start_ts + interval
        subset = X.loc[start_ts:end_ts]
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method
        ]
        values.append(np.concatenate(aggregated))
        index.append(start_ts)
        start_ts = end_ts

    return np.asarray(values), np.asarray(index)

X_tsa, index = time_segments_aggregate(df, interval=1000000000, time_column='timestamp')

In [8]:
imp = SimpleImputer()
X_imp = imp.fit_transform(np.asarray(df))

ValueError: Cannot use mean strategy with non-numeric data. Received datatype :O.

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1)) ## for the gradients to converge faster == try 0-1 StandardScaler()
X_scl = scaler.fit_transform(X_imp)

In [3]:
# To find the correlation among
# the columns using pearson method
df.corr(method ='pearson')

,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
timestamp,1.000000,-0.236354,0.241988,-0.225518,-0.236343,0.077486,-0.958600,0.693874,0.221608,-0.232588,...,-0.454911,-0.457006,NaN,-0.470718,-0.541751,-0.473398,-0.021117,NaN,-0.019010,NaN
FIT101,-0.236354,1.000000,-0.712931,0.968146,0.341624,-0.054708,0.234835,-0.252807,-0.287970,0.344309,...,0.472714,0.473927,NaN,0.479682,0.387051,0.480054,0.073130,NaN,0.067238,NaN
LIT101,0.241988,-0.712931,1.000000,-0.703150,-0.723447,-0.029932,-0.250289,0.387364,0.133196,-0.729782,...,-0.534645,-0.535860,NaN,-0.544474,-0.415582,-0.544942,-0.076062,NaN,-0.070880,NaN
MV101,-0.225518,0.968146,-0.703150,1.000000,0.342197,-0.052457,0.222083,-0.247490,-0.270743,0.344980,...,0.454185,0.455418,NaN,0.461018,0.369489,0.461381,0.072729,NaN,0.067175,NaN
P101,-0.236343,0.341624,-0.723447,0.342197,1.000000,0.074665,0.235398,-0.449555,0.059288,0.992811,...,0.485733,0.486396,NaN,0.498403,0.393546,0.499262,0.022871,NaN,0.027325,NaN
P102,0.077486,-0.054708,-0.029932,-0.052457,0.074665,1.000000,-0.088168,0.015817,-0.038488,0.105161,...,0.037516,0.038070,NaN,0.035176,0.058552,0.034680,0.014432,NaN,0.012771,NaN
AIT201,-0.958600,0.234835,-0.250289,0.222083,0.235398,-0.088168,1.000000,-0.699643,-0.160085,0.231409,...,0.463734,0.464874,NaN,0.482081,0.530025,0.484964,0.018411,NaN,0.016482,NaN
AIT202,0.693874,-0.252807,0.387364,-0.247490,-0.449555,0.015817,-0.699643,1.000000,-0.058406,-0.450306,...,-0.365463,-0.367228,NaN,-0.378331,-0.429596,-0.380315,-0.025220,NaN,-0.024619,NaN
AIT203,0.221608,-0.287970,0.133196,-0.270743,0.059288,-0.038488,-0.160085,-0.058406,1.000000,0.060767,...,-0.601001,-0.601536,NaN,-0.605423,-0.511738,-0.605266,-0.011532,NaN,-0.008359,NaN
FIT201,-0.232588,0.344309,-0.729782,0.344980,0.992811,0.105161,0.231409,-0.450306,0.060767,1.000000,...,0.486181,0.486879,NaN,0.498953,0.394593,0.499811,0.029448,NaN,0.034234,NaN


In [4]:
# To find the correlation among
# the columns using kendall method
df.corr(method ='kendall')

/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/scipy/stats/stats.py:4814: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(var) / np.sqrt(2)))


,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
timestamp,1.000000,-0.142440,0.152781,-0.188197,-0.192974,0.063267,-0.845585,0.686289,0.010165,-0.050094,...,-0.093252,-0.373144,NaN,-0.185354,-0.364002,-0.214619,-0.360941,NaN,-0.015522,NaN
FIT101,-0.142440,1.000000,-0.409406,0.715979,0.249655,-0.038642,0.124818,-0.181496,-0.117115,0.053383,...,0.119057,0.345989,NaN,0.181182,0.177528,0.184629,0.133956,NaN,0.048947,NaN
LIT101,0.152781,-0.409406,1.000000,-0.558080,-0.572209,-0.035799,-0.130914,0.309415,-0.084653,-0.280590,...,-0.167693,-0.412525,NaN,-0.209494,-0.181076,-0.208712,-0.138172,NaN,-0.061870,NaN
MV101,-0.188197,0.715979,-0.558080,1.000000,0.346245,-0.053179,0.162024,-0.244889,-0.150419,0.078129,...,0.157400,0.464492,NaN,0.243056,0.229224,0.247366,0.168520,NaN,0.067577,NaN
P101,-0.192974,0.249655,-0.572209,0.346245,1.000000,0.074665,0.166913,-0.447709,0.248433,0.709596,...,0.187464,0.486396,NaN,0.240948,0.236887,0.242874,0.168872,NaN,0.027325,NaN
P102,0.063267,-0.038642,-0.035799,-0.053179,0.074665,1.000000,-0.064055,0.012894,-0.040385,0.154430,...,-0.012456,0.038070,NaN,-0.035486,0.057843,-0.037823,0.017053,NaN,0.012771,NaN
AIT201,-0.845585,0.124818,-0.130914,0.162024,0.166913,-0.064055,1.000000,-0.662337,0.017809,0.043274,...,0.137670,0.338911,NaN,0.205579,0.400368,0.235057,0.428966,NaN,0.014683,NaN
AIT202,0.686289,-0.181496,0.309415,-0.244889,-0.447709,0.012894,-0.662337,1.000000,-0.230298,-0.229416,...,-0.056434,-0.358234,NaN,-0.146728,-0.334153,-0.171355,-0.325073,NaN,-0.029311,NaN
AIT203,0.010165,-0.117115,-0.084653,-0.150419,0.248433,-0.040385,0.017809,-0.230298,1.000000,0.242169,...,-0.149369,-0.337749,NaN,-0.046275,-0.216965,-0.042139,-0.163279,NaN,0.002088,NaN
FIT201,-0.050094,0.053383,-0.280590,0.078129,0.709596,0.154430,0.043274,-0.229416,0.242169,1.000000,...,0.161189,0.348700,NaN,0.246827,0.143547,0.244559,0.095012,NaN,0.004169,NaN
